In [1]:
import copy
import glob
import json
import os
import random
import re
from collections import Counter
from random import shuffle

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import fastparquet
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
dataset_folder='./../dataset/saved/'

In [2]:
df_train = pd.read_pickle(dataset_folder+'train_2_ohe401')
df_test = pd.read_pickle(dataset_folder+'test_2_ohe401')
df_valid = pd.read_pickle(dataset_folder+'valid_2_ohe401')

In [3]:
df_train_meta, df_train_num = df_train[['correct', 'group']], df_train.drop(columns=['correct', 'group', 'name', 'firstChar'])
del df_train
df_test_meta, df_test_num = df_test[['correct', 'group']], df_test.drop(columns=['correct', 'group', 'name', 'firstChar'])
del df_test
df_valid_meta, df_valid_num = df_valid[['correct', 'group']], df_valid.drop(columns=['correct', 'group', 'name', 'firstChar'])
del df_valid

In [4]:
print(df_train_num.dtypes)
print(df_train_num.shape)
print('--------')
print(df_train_meta.dtypes)
print(df_train_meta.shape)

name_Abstract                     Sparse[int64, 0]
name_Action                       Sparse[int64, 0]
name_Actions                      Sparse[int64, 0]
name_All                          Sparse[int64, 0]
name_An                           Sparse[int64, 0]
                                        ...       
insideStatement2_return           Sparse[int64, 0]
insideStatement2_switch_branch    Sparse[int64, 0]
insideStatement2_ternary_else     Sparse[int64, 0]
insideStatement2_try              Sparse[int64, 0]
insideStatement2_while_body       Sparse[int64, 0]
Length: 740, dtype: object
(334350, 740)
--------
correct    int64
group      int64
dtype: object
(334350, 2)


In [ ]:
poly = PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)
print('go')
train_intr = poly.fit_transform(df_train_num)
del df_train_num
print('1')
test_intr = poly.fit_transform(df_test_num)
del df_test_num
print('2')
valid_intr = poly.fit_transform(df_valid_num)
del df_valid_num
print('3')

go


In [ ]:
print(f'train ########## ------> : {train_intr.shape}')
# print(f'test ########## ------> : {test_intr.shape}')
print(f'valid ########## ------> : {valid_intr.shape}')

In [ ]:
#train_cols = df_train_num.columns.tolist()
train_cols = []
train_cols = train_cols + [f'intr_{i}' for i in range(train_intr.shape[1] - len(train_cols))]

#test_cols = df_test_num.columns.tolist()
test_cols = []
test_cols = test_cols + [f'intr_{i}' for i in range(test_intr.shape[1] - len(test_cols))]

#valid_cols = df_valid_num.columns.tolist()
valid_cols = []
valid_cols = valid_cols + [f'intr_{i}' for i in range(valid_intr.shape[1] - len(valid_cols))]

print(train_cols)
#valid_cols


In [ ]:
df_train_num_intr = pd.DataFrame.sparse.from_spmatrix(train_intr, columns=train_cols)
del train_intr
df_test_num_intr = pd.DataFrame.sparse.from_spmatrix(test_intr, columns=test_cols)
del test_intr
df_valid_num_intr = pd.DataFrame.sparse.from_spmatrix(valid_intr, columns=valid_cols)
del valid_intr

In [ ]:
df_train_num_intr.dtypes
#df_valid_num_intr.dtypes

In [ ]:
df_train_intr = pd.concat([df_train_meta, df_train_num_intr], axis=1)
del df_train_num_intr
df_test_intr = pd.concat([df_test_meta, df_test_num_intr], axis=1)
del df_test_num_intr
df_valid_intr = pd.concat([df_valid_meta, df_valid_num_intr], axis=1)
del df_valid_num_intr

In [ ]:
s_tr_intr = df_train_intr.drop(columns=['group']).sum().div(df_train_intr.shape[0]).mul(100)
#s_tr_intr = df_valid_intr.drop(columns=['group']).sum().div(df_valid_intr.shape[0]).mul(100)

In [ ]:
s_tr_intr.sort_values(ascending=[False])

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(np.arange(s_tr_intr.shape[0]), s_tr_intr.sort_values(ascending=[False]).to_list())
plt.yscale('log')
plt.show()

In [ ]:
selector = SelectKBest(score_func=chi2, k=6*10**3)
selector.fit(df_train_intr.drop(columns=['correct', 'group']), df_train_intr['correct'])

In [ ]:
cols = list(selector.get_feature_names_out()) + ['correct', 'group']
df_train_intr_s = df_train_intr[cols]
del df_train_intr
df_test_intr_s = df_test_intr[cols]
del df_test_intr
df_valid_intr_s = df_valid_intr[cols]
del df_valid_intr

del selector

In [ ]:
df_train_intr_s.dtypes

In [ ]:
s_tr_intr_s = df_train_intr_s.drop(columns=['group']).sum().div(df_train_intr_s.shape[0]).mul(100)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(np.arange(s_tr_intr_s.shape[0]), s_tr_intr_s.sort_values(ascending=[False]).to_list())
plt.yscale('log')
plt.show()

plt.figure(figsize=(20, 10))
plt.plot(np.arange(s_tr_intr_s.shape[0]), s_tr_intr_s.to_list())
plt.yscale('log')
plt.show()

In [ ]:
df_train_intr_s.to_pickle(dataset_folder+'train_ohe401_intr2_2k')
df_test_intr_s.to_pickle(dataset_folder+'test_ohe401_intr2_2k')
df_valid_intr_s.to_pickle(dataset_folder+'valid_ohe401_intr2_2k')